# Self-Supervision

- Train a deep network using the self-supervised learning framework.

-----

## Load Packages

In [ ]:
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2
%cd ..

In [ ]:
# Load some packages
import hydra
from omegaconf import OmegaConf
import wandb
import pprint

# custom package
from run_train import check_device_env
from run_ssl_train import prepare_and_run_ssl_train

---

## Specify the dataset, model, and train setting

In [ ]:
project = 'caueeg-ssl'
data_cfg_file = 'caueeg-dementia'
train_cfg_file = 'ddp-amp-gradclip'
ssl_cfg_file = 'byol'
model_cfg_file = '2D-ResNet-50'
# device = 'cuda:0'

---

## Initializing configurations using Hydra

In [ ]:
with hydra.initialize(config_path='../config'):
    add_configs = [f"data={data_cfg_file}", 
                   f"++data.seq_length=8000",
                   f"++data.input_norm='datapoint",
                   f"++data.awgn=0.05",
                   f"++data.mgn=0.05",
                   f"++data.channel_dropout=0.1",
                   f"++data.crop_multiple=2",
                   f"train={train_cfg_file}", 
#                   f"+train.device={device}", 
                   f"+train.project={project}",
                   f"model={model_cfg_file}",
                   f"++train.save_model=True",
                   f"ssl={ssl_cfg_file}",
                   f"++model.criterion=multi-bce",]
    
    cfg = hydra.compose(config_name='default', overrides=add_configs)
    
config = {**OmegaConf.to_container(cfg.data), 
          **OmegaConf.to_container(cfg.train),
          **OmegaConf.to_container(cfg.model),
          **OmegaConf.to_container(cfg.ssl)}

check_device_env(config)
pprint.pprint(config)

## Train

In [ ]:
prepare_and_run_ssl_train(rank=None, world_size=None, config=config)